In [ ]:
from utils import prepare_image_forgery_dataset

data_directory = '../data'

IMG_SIZE=(128, 128)

X_train, X_test, y_train, y_test = prepare_image_forgery_dataset(total_number=6000, test_size=0.2, random_state=42, 
                                                                 data_directory=data_directory, img_size=IMG_SIZE)

In [ ]:
from utils import display_images

classnames = ['Authentic', 'Tampared']

display_images(X_train, y_train, classnames=classnames)

In [2]:
from models.cnn import CNNImageForgeryDetector
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Input, Dropout
from tensorflow.keras.optimizers import Adam

IMG_SIZE=(128, 128)

model = CNNImageForgeryDetector(compression_quality=80, learning_rate=0.0001, img_size=IMG_SIZE, epochs=15)

def create_model(self):
    model = Sequential([
        Input(shape=self.img_shape),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),  # Add a dropout layer with a rate of 0.25
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),   # Add another dropout layer with a rate of 0.5
        Dense(2, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='binary_crossentropy',
                      metrics=['accuracy'])
    return model

model.model = create_model(model)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
history = model.fit(X_train, y_train, X_val, y_val)

 24/122 [====>.........................] - ETA: 1:28 - loss: 0.4544 - accuracy: 0.7812

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from utils import print_model_performance_metrics

print_model_performance_metrics(y_test, y_pred)